Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

W szczególności zwróć uwagę, że usupełniłeś wszystkie miejsca `YOUR CODE HERE`, `WPISZ TWÓJ KOD TUTAJ`, "YOUR ANSWER HERE" lub "WPISZ TWOJĄ ODPOWIEDŹ TUTAJ".

### Zaawansowane Przetwarzanie Języka Naturalnego
# Laboratorium 2

Pobierz zbiór danych Amazon "Musical Instruments" z [tej](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz) strony internetowej, a następnie wczytaj go poniższym kodem. Zwróć uwagę na wymaganą lokalizację pliku, tj. dwa katalogi wyżej - wynika to ze struktury plików w sprawdzarce, przepraszam za niedogodność.



In [1]:
from collections import defaultdict, Counter
import time
import random
import torch
import json
 
x_text = []
y = []
with open('../../Musical_Instruments_5.json') as f:
    for line in f:
        data = json.loads(line)
        x_text.append(data['reviewText'].lower().strip())
        y.append(int(data['overall']))

## Zadanie 1 - przygotowanie danych
W załadowanych listach `x_text` oraz `y` znajdują się odpowiednio teksty kolejnych opinii oraz oznaczenia klas. Klasą w tym wypadku jest liczba gwiazdek (ocena) produktu towarzysząca opinii. Zadanie klasyfikacji polega na przewidzeniu oceny na podstawie opinii pozostawionej w portalu.

Aby zmniejszyć wymagania obliczeniowe do dalszych eksperymentów, ograniczymy zbiór danych jedynie do pierwszego tysiąca opinii.



In [2]:
x_text = x_text[:1000]
y = y[:1000]
train_end_idx=int(0.9 * len(y))

Jedną z użytecznych operacji przygotowania tekstu do konstrukcji klasyfikatora jest zastąpienie poszczególnych tokenów ich indeksami. Chociaż w praktyce ten proces często następuje dopiero po szeregu etapów przetwarzania tekstu takich jak tokenizacja, lematyzacja czy stemming - w tym ćwiczeniu wyodrębnimy tokeny rozdzielając tekst znakiem spacji.

Klasyfikator powinien obsługiwać także słowa, które nie występowały w zbiorze uczącym. Podstawową techniką obsługi takich słów jest wprowadzenie specjalnego tokenu UNK, obsługującego nieznane słowa. W tym celu usuwa się ze zbioru danych pewną liczbę najrzadszych słów i zastępuje się je tokenami UNK.

Zbuduj słownik `w2i` mapujący tokeny na kolejne indeksy tj. liczby naturalne. Pomiń tokeny występujące w zbiorze uczącym 5 lub mniej razy.



In [3]:
from itertools import chain

w2i = defaultdict(lambda: len(w2i))
UNK = w2i["<unk>"] #Przypisz indeks tokenowi UNK

words = list(chain(*[sentence.split(" ") for sentence in x_text]))
counter = Counter(words)
for token in words:
    if token not in w2i.keys() and counter[token] > 5:
        w2i[token]
        

n_words = len(w2i)

Po zbudowaniu słownika `w2i`, przekonwertujmy nasz zbiór danych z listy słów na listę indeksów słów. Od razu podzielimy zbiór na część uczącą i część testową, a także przekonwertujemy klasy na indeksy klas.

In [4]:
w2i = defaultdict(lambda: UNK, w2i) # Domyślną wartością słownika jest UNK, 
         #chociaż w2i będzie zawierał wpisy do wszystkich słów to nowym tokenom będzie przypisywał indeks UNK
class2i = defaultdict(lambda: len(class2i))
        # mapuj klasy na indeksy klas
    
def read_dataset(start_idx,end_idx):
    for i, text in enumerate(x_text[start_idx:end_idx]):
        yield ([w2i[x] for x in text.split(" ")], class2i[y[i]])
        
train = list(read_dataset(0, train_end_idx))
dev = list(read_dataset(train_end_idx, len(y)))
n_class = len(class2i)

In [5]:
print(n_words, n_class)

1389 5


## Zadanie 2 - pierwszy model klasyfikacji tekstu w PyTorch
Podstawową strukturą danych w PyTorch jest tensor, na którym możesz wykonywać analogiczne operacje jak na macierzach `numpy`. Podstawową metodą stworzenia tensora jest wywołanie konstruktora `torch.tensor` na liście liczb. Istnieją także inne konstruktory tensorów, analogiczne do `numpy`. Można też na nich operować za pomocą standardowych operatorów, indeksowania, i odpowiedników innych funkcji znanych z `numpy`.



In [6]:
print(torch.tensor([1,2,3]))
print(torch.rand( (3,3) ))
print(torch.ones( (3,3) ))
print(2 * torch.ones( (3,3) ))

tensor([1, 2, 3])
tensor([[0.5711, 0.9193, 0.3689],
        [0.4934, 0.0594, 0.3880],
        [0.1660, 0.3467, 0.7812]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])


Dlaczego więc korzystamy z PyTorch, a nie z biblioteki `numpy` skoro tensory wydają się mieć analogiczną funkcjonalność do poznanych uprzednio macierzy? Powodów jest oczywiście wiele, m.in. możliwość przeniesienia obliczeń na kartę graficzną (technologia CUDA), ale z punktu widzenia naszego ćwiczenia kluczowa jest funkcjonalność automatycznego liczenia gradientów. W przypadku konstrukcji sieci neuronowej czy modelu liniowego, w PyTorch nie jest konieczne samodzielne wyprowadzanie i implementowanie gradientu, gdyż biblioteka zrobi to za nas automatycznie.

Wyznaczanie gradientów odbywa się za pomocą algorytmu wstecznej propagacji, który ma dwie fazy: *forward* i *backward*. Faza *forward* polega na policzeniu wyniku funkcji, a faza *backward* wyznacza gradienty wszystkich jej parametrów.

W celu poznania tej funkcjonalności policzymy pochodne cząstkowe prostej funkcji kwadratowej:
$$result = x_1^2 + x_2^2+ x_3^2$$
której pochodne cząstkowe mają postać $2x_i$.

Rozpocznijmy implementacje tej funkcji od stworzenia 3-elementowego wektora zmiennych `x`.



In [7]:
x = torch.tensor([1.,2.,3.], requires_grad=True)

Jak pewnie zauważyłeś, w konstruktorze użyliśmy dodatkowego parametru `requires_grad`. Domyślnie wykonanie operacji na dowolnym tensorze nie traktuje się jako części fazy `forward`, gdyż nie do wszystkich tensorów użytych w kodzie będziemy potrzebować wartości pochodnych. Aby zasygnalizować, że dla danej zmiennej konieczne jest zapisywanie informacji o wykonywanych na niej operacjach, należy ustawić wartość jej parametru `requires_grad` na `True`.



In [8]:
print(x.requires_grad)

True


Przejdźmy do policzenia wartości wyżej zdefiniowanej funkcji.

In [9]:
result = (x**2).sum()

Tensory posiadają parametr `.grad`, który przechowuje informacje o wyznaczonym gradiencie.

In [10]:
print(x.grad)

None


W tej chwili, pomimo obliczenia wartości zmiennej `result`, wartość gradientu nie jest policzona, gdyż nie poinformowaliśmy biblioteki o zakończeniu fazy `forward` i konieczności wykonania fazy `backward`. Możemy to zrobić poprzez wykonanie funkcji `backward()` na obliczonej wartości funkcji (funkcję tę można wywołać tylko na skalarnym wyniku!).




In [11]:
result.backward()

In [12]:
x.grad

tensor([2., 4., 6.])

Zwróć uwagę, że wektor `x.grad`, zgodnie z naszymi oczekiwaniami, zawiera wartości pochodnej cząstkowej tj. `2x`. Spróbujmy jeszcze raz, licząc pochodną po logarytmie z `result`.

In [13]:
result2 = torch.log(result)

In [14]:
result2.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

Niestety operacja się nie powiodła. Przed wykonaniem kolejnej fazy *backward* należy - upraszczając - wykonać fazę *forward*. Nasze poprzednie operacje konstruowały fazę *forward* od parametrów `x` aż do zmiennej z wynikiem, jednakże przy wykonaniu fazy *backward* została zwolniona pamięć przechowująca informacje o kolejno wykonywanych operacjach na tych zmiennych (graf obliczeń). Kolejna operacja została wykonana bezpośrednio na tensorze `result`, konstruując fazę *forward* od `result` do `result2`, jednak zabrakło grafu obliczeń od parametrów `x`.

Uruchomienie poniższego kodu, z operacjami rozpoczynającymi się od `x`, zakończy się obliczeniem gradientu z sukcesem.



In [15]:
result = (x**2).sum()
result2 = torch.log(result)
result2.backward()
print(x.grad)

tensor([2.1429, 4.2857, 6.4286])


Sprawdźmy poprawność uzyskanego wyniku. Zmienna $result= 1^2+2^2+3^2=14$, a pochodna z logarytmu naturalnego to $\frac{1}{x}$. W związku z tym:
$$\frac{\partial }{\partial x_1} \log result = \frac{1}{result} \cdot \frac{\partial }{\partial x_1} result = \frac{1}{result} 2x_1 $$
Przy naszych wartościach $x$ równa się to $\frac{1}{14}\cdot 2 = 0,1428$. Łatwo zauważyć, że wynik znajdujący się w tensorze `x.grad` jest błędny, a konkretnie za duży o 2 jednostki.

Stało się tak dlatego, że gradient z kolejnych faz `backward` jest akumulowany w parametrze `.grad` (poprzednia wartość policzonej pochodnej cząstkowej wynosiła właśnie 2). Takie zachowanie biblioteki może być bardzo użyteczne w sytuacji gdy chcemy w zmiennej zagregować gradienty funkcji celu liczonych na kolejno przetwarzanych instancjach lub przy treningu modelu z wieloma funkcjami celu; tutaj jednak doprowadziło to do błędnego wyniku. Z tego powodu bardzo ważne jest pamiętanie o wyzerowaniu wartości gradientów przed przystąpieniem do kolejnych obliczeń.



In [16]:
x.grad.zero_()

tensor([0., 0., 0.])

In [17]:
result = (x**2).sum()
result2 = torch.log(result)
result2.backward()
print(x.grad)

tensor([0.1429, 0.2857, 0.4286])


Zwróć uwagę na konwencję biblioteki PyTorch - jeśli nazwa funkcji zakończona jest podkreślnikiem to taka operacja jest wykonywana `in-place`. (np. `x.add(5)` - `x` nadal ma stałą wartość, `x.add_(5)` wartość `x` zwiększono o 5).

Zaimplementujmy podstawowy algorytm uczący w PyTorch. Będzie to prosta sieć neuronowa składająca się z:
- macierzy zanurzeń $C$, przetwarzającej indeksy słów na odpowiednie reprezentacje wektorowe, 
- operacji uśredniania tych zanurzeń do jednego zanurzenia (average pooling over time) 
- oraz jednej warstwy liniowej (softmax) zwracającej wynik.

Algorytmem uczącym będzie SGD optymalizujące entropię krzyżową, czyli dla kolejnych instancji uczących będziemy wykonywać:
$$parametry = parametry - \eta \nabla f\_celu$$
Implementacja ta będzie wyjątkowo prosta, gdyż gradient funkcji celu ($\nabla f\_celu$) zostanie obliczony automatycznie przez PyTorch. Ponadto entropia krzyżowa jest już zaimplementowana w PyTorch `F.cross_entropy(logits, target)`. Zwróć uwagę, że argumentem tej funkcji są wartości logitów (nie trzeba implementować funkcji softmax przetwarzającej wartości logitów na prawdopodobieństwa).

**UWAGA** W implementacji nie należy używać gotowych implementacji SGD czy warstw sieci neuronowych w PyTorch.



Pierwszym krokiem w implementacji będzie zaimplementowanie samego modelu. Należy zainicjalizować macierz $C$ przechowującą w wierszach zanurzenia dla kolejnych słów (liczba słów to `n_words`, wymiarowość zanurzenia określ na 20) oraz macierz $W$ przechowującą parametry warstwy liniowej, zwracającej wartości logitów dla każdej z klas (liczba klas to `n_class`). Macierz $W$ w dodatkowej kolumnie powinna też przechowywać wartości wyrazów wolnych (bias). Wartości zainicjalizuj losowo `torch.rand`. Pamiętaj, że dla tych macierzy będziesz potrzebował wyznaczyć potem wartości gradientów.



In [18]:
EMBEDDING_SIZE = 20

def initialize_tensors(size):
    C = torch.rand((n_words, size), requires_grad=True)
    W = torch.rand((n_class, size + 1), requires_grad=True)
    return C, W

C, W = initialize_tensors(EMBEDDING_SIZE)

Zaimplementuj funkcję `simple_model`, której argumentem będzie instancja testowa (jest to więc lista indeksów słów występujących w tekście), a na której wyjściu będzie wektor `n_class`-elementowy zawierający obliczone wartości logitów.

In [19]:
def simple_model(x):
    doc_embedding = torch.mean(C[x], dim=0)
    doc_with_bias = torch.cat([torch.ones(1),doc_embedding]) # Skonkatenowanie 1 z uzyskaną reprezentacją (bias)
    return W @ doc_with_bias # Obliczenie wartości logitów (tj. warstwa liniowa)

Zaimplementuj algorytm SGD w poniższej pętli. Pętla ta iteruje po zbiorze uczącym oraz dla każdej instancji oblicza wartość funkcji celu. Twoje zadania:
- Policz gradienty (faza *backward*)
- Zaimplementuj aktualizacje $W$ i $C$ wg. wzoru na SGD. Operacje modyfikujące $W$ i $C$ musisz wykonać w środku klauzuli `with torch.no_grad():`, aby nie śledzić z tych operacji gradientów.
- Pamiętaj o wyczyszczeniu gradientów (zarówno w $W$ jak i $C$)



In [25]:
import torch.nn.functional as F
epochs = 5
eta = 0.5  # prędkość uczenia
C, W = initialize_tensors(EMBEDDING_SIZE)

for i in range(epochs):
    random.shuffle(train)
    train_loss = 0.0
    for words, tag in train:
        pred = simple_model(words)
        loss = F.cross_entropy(pred.reshape(1,-1), torch.tensor(tag).reshape(1))
        # Loss zawiera wartość funkcji celu dla przykładu, wykonaj backpropagation
        loss.backward()
        
        with torch.no_grad():
            train_loss += loss

            W -= eta * W.grad
            C -= eta * C.grad

            W.grad.zero_()
            C.grad.zero_()
    print("iter %r: avg. train loss=%.4f" % (i, train_loss / len(train)))

iter 0: avg. train loss=1.1355
iter 1: avg. train loss=0.9601
iter 2: avg. train loss=0.9460
iter 3: avg. train loss=0.9054
iter 4: avg. train loss=0.8783


**Ćwiczenia**
- Dlaczego w bibliotekach do głębokiego uczenia maszynowego, takich jak PyTorch, implementuje się funkcje entropii krzyżowej tak, aby przyjmowała na wejście wartości logitów zamiast prawdopodobieństw z softmax?
- Na wykładzie pokazywaliśmy warstwę zanurzeń jako warstwę mnożącą macierz $C$ przez wektor "1 z n", można ją jednak także zaimplementować jako operację odczytu odpowiedniego wiersza z macierzy. Jakie są wady i zalety obu tych sposobów implementacji?

Odpowiedź na pierwszą kropkę umieść poniżej.



WPISZ TWOJĄ ODPOWIEDŹ TUTAJ

## Zadanie 3 - wykorzystanie nn.Module


PyTorch jako biblioteka do głębokiego uczenia maszynowego oferuje nam kilka udogodnień w implementowaniu modeli uczących się, aby jeszcze bardziej uprościć ich implementację. Większość z tych udogodnień związanych z jest z reprezentowaniem modeli uczących się jako obiektów dziedziczących po `torch.nn.Module`. W obiekcie takim powinniśmy zaimplementować co najmniej konstruktor, inicjalizujący parametry modelu ($W$ i $C$), oraz funkcję `forward` obliczającą wynik modelu (we wcześniejszym zadaniu nazywaliśmy ją `simple_model`). Ponadto moduł `torch.nn` oferuje gotowe implementacje zarówno warstwy liniowej jak i warstwy zanurzeń.

Przeanalizuj poniższą implementację modelu z poprzedniego zadania.



In [21]:
class SimpleModel(torch.nn.Module):
    def __init__(self, n_words, emb_size, n_class):
        super(SimpleModel, self).__init__()
        self.embedding = torch.nn.Embedding(n_words, emb_size)
        self.linear = torch.nn.Linear(in_features=emb_size, out_features=n_class, bias=True)
        torch.nn.init.uniform_(self.embedding.weight, -0.25, 0.25)
        torch.nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, words):
        emb = self.embedding(words)                 
        h = emb.mean(dim=0)                         
        h = torch.reshape(h, (1,-1))
        out = self.linear(h)              
        return out


Oprócz tego, że uzyskaliśmy elegancki obiekt reprezentujący nasz model, nie wydaje się by powyższa implementacja była krótsza czy prostsza od tej, którą uzyskaliśmy w poprzednim zadaniu bez dobrodziejstw `nn.Module`. Co zatem zyskaliśmy?

Przy implementacji modeli z dużą liczbą warstw, szczególnie uciążliwe byłoby implementowanie kolejnych linijek kodu zerujących gradienty wszystkich macierzy wag, oraz wykonywanie na nich kroków algorytmu SGD. W naszej implementacji każda macierz parametrów to dwie linijki kodu! Jednak modele dziedziczące po `torch.nn.Module` i stworzone poprzez dedykowane warstwy neuronowe posiadają gotową funkcję `parameters()` zwracającą kolejne macierze parametrów modelu.



In [87]:
model = SimpleModel(n_words, EMBEDDING_SIZE, n_class)
print([i for i in model.parameters()])

[Parameter containing:
tensor([[ 0.1609,  0.0989, -0.0518,  ..., -0.2365,  0.0248, -0.0152],
        [-0.2254,  0.0394, -0.0669,  ..., -0.1842,  0.0470,  0.2063],
        [-0.1310, -0.0271, -0.2397,  ..., -0.1859, -0.1206, -0.2478],
        ...,
        [-0.0933,  0.1210,  0.1976,  ..., -0.2445, -0.0927, -0.1077],
        [ 0.1012,  0.0211, -0.1043,  ...,  0.1180,  0.1067, -0.1309],
        [-0.0363,  0.2183,  0.1583,  ...,  0.0137, -0.0222,  0.1815]],
       requires_grad=True), Parameter containing:
tensor([[ 0.1161,  0.0693,  0.2821, -0.2650, -0.4008, -0.2819, -0.1938,  0.2585,
          0.2546,  0.4209,  0.4725, -0.1389, -0.1003,  0.1012,  0.1961,  0.2380,
          0.4339, -0.1890,  0.2815, -0.3693],
        [-0.3983,  0.3975,  0.3072,  0.2348,  0.3773, -0.1205,  0.2124, -0.2139,
          0.1748, -0.3769, -0.4518,  0.1269,  0.3118, -0.2520, -0.1354, -0.0948,
          0.1195, -0.1540,  0.0308, -0.0587],
        [-0.4726,  0.0852,  0.2667, -0.4116,  0.4858, -0.4603,  0.2669,  0.47

Jest to niezwykle wygodne, bo implementacja algorytmu SGD może przeiterować po tej liście parametrów i dla każdej z nich wykonać aktualizację ich wartości. Fakt, że taka lista jest tworzona automatycznie pozbawia nas ryzyka, że zwyczajnie o którejś macierzy parametrów czy wektorze wyrazów wolnych najzwyczajniej zapomnimy. 

Podobnie można zaimplementować pętlę zerującą gradienty wszystkich parametrów. Modele oferują nawet gotową taką funkcję `model.zero_grad()`, która iteruje po parametrach zerując ich gradienty. 

Zmodyfikuj implementację SGD z poprzedniego zadania, tak aby wykorzystywała `zero_grad()` i `parameters()`.



In [24]:
epochs = 5
eta = 0.5  # prędkość uczenia

for i in range(epochs):
    random.shuffle(train)
    train_loss = 0.0
    for words, tag in train:
        pred = model(torch.tensor(words))
        loss = F.cross_entropy(pred.reshape(1,-1), torch.tensor(tag).reshape(1))
        # Loss zawiera wartość funkcji celu dla przykładu, wykonaj backpropagation
        loss.backward()
        with torch.no_grad():
            train_loss += loss

            for i in model.parameters():
                i -= eta * i.grad

            model.zero_grad()
        
    print("iter %r: avg. train loss=%.4f" % (i, train_loss / len(train)))

iter Parameter containing:
tensor([ 1.9933,  0.4218,  1.2478, -1.7495, -1.9941], requires_grad=True): avg. train loss=0.9863
iter Parameter containing:
tensor([ 2.5760, -0.3060,  1.5082, -1.6670, -2.1920], requires_grad=True): avg. train loss=0.9618
iter Parameter containing:
tensor([ 2.7519, -0.4110,  1.4690, -1.7125, -2.1782], requires_grad=True): avg. train loss=0.9248
iter Parameter containing:
tensor([ 2.5992, -1.0454,  1.1576, -1.6292, -1.1630], requires_grad=True): avg. train loss=0.8918
iter Parameter containing:
tensor([ 1.6563, -0.1511,  1.8729, -1.4825, -1.9764], requires_grad=True): avg. train loss=0.8778


Dodatkowo moduł `torch.nn` oferuje także od razu zaimplementowane optymalizatory, w tym SGD. W konstruktorze optymalizatora należy podać listę optymalizowanych przez niego parametrów, a następnie wywołać na nim procedurę `step()` wykonującą krok algorytmu optymalizacyjnego tj. aktualizację wartości zmiennych przy użyciu gradientu. W tej sytuacji nie musisz się martwić o umieszczanie kodu zmieniającego parametry w `with torch.no_grad()` - optymalizator sam to zrobi! Optymalizator również oferuje funkcję `zero_grad()`, zerującą gradienty zmiennych wskazanych do optymalizacji.

Zmodyfikuj kod z poprzedniego zadania, tak aby wykorzystywał optymalizator SGD zaimplementowany w `torch.optim`.



In [90]:
epochs = 5
eta = 0.5  # prędkość uczenia

for i in range(epochs):
    random.shuffle(train)
    train_loss = 0.0
    optimizer = torch.optim.SGD(model.parameters(), lr=eta)
    for words, tag in train:
        pred = model(torch.tensor(words))
        loss = F.cross_entropy(pred.reshape(1,-1), torch.tensor(tag).reshape(1))
        train_loss += loss
        # Loss zawiera wartość funkcji celu dla przykładu, wykonaj backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        
    print("iter %r: avg. train loss=%.4f" % (i, train_loss / len(train)))

iter 0: avg. train loss=0.5481
iter 1: avg. train loss=0.5378
iter 2: avg. train loss=0.4791
iter 3: avg. train loss=0.4586
iter 4: avg. train loss=0.4480


**Ćwiczenia**
- Przeanalizuj dokładnie powyższy kod i przechodząc linia po linii, wyjaśnij co one robią z punktu widzenia treningu modelu.
- Zastanów się jak wyglądałaby Twoja własna implementacja klasy `optim.SGD`.
- Prześledź jeszcze raz implementację modelu neuronowego - pewnie w niedługim czasie będziesz implementował znacznie bardziej skomplikowane modele, tym bardziej warto je dobrze prześledzić!
- Czym różni się zaimplementowana architektura od głębokiej sieci uśredniającej?
- Na wykładzie korzystaliśmy z macierzy zanurzeń w modelach języka. Tutaj warstwa zanurzeń pojawiła się bezpośrednio w modelu klasyfikacji. Czy w uzyskanych w ten sposób zanurzeniach (zakładając dobry dobór hiperparamerów, dodanie regularyzacji itd.) zaobserwowalibyśmy podobne zależności jak te uzyskane za pomocą modelu języka? Jeśli nie, obserwacji jakich zależności między słowami spodziewałbyś się w tej reprezentacji? Skąd biorą się różnice?

Odpowiedź na ostatnią kropkę umieść poniżej.





WPISZ TWOJĄ ODPOWIEDŹ TUTAJ

# Zadanie 4
Wykorzystując wiedzę z poprzedniego zadania zaimplementuj prostą architekturę splotową do klasyfikacji tekstu i wytrenuj ją. Do jej wykonania może być przydatna klasa `torch.nn.Conv1d` i funkcja `torch.nn.ReLU` (zapoznaj się z ich dokumentacją w Internecie). Jako funkcji redukcji użyj funkcji maksimum (over time).



In [91]:
class CNN(torch.nn.Module):
    def __init__(self, n_words, emb_size, num_filters, window_size, ntags):
        super(CNN, self).__init__()
        self.embedding = torch.nn.Embedding(n_words, emb_size)
        self.conv_layer = torch.nn.Conv1d(in_channels=emb_size, out_channels=num_filters, kernel_size=window_size)
        self.linear = torch.nn.Linear(in_features=num_filters, out_features=ntags, bias=True)

    def kmax_pooling(self, x, dim, k):
        index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
        return x.gather(dim, index)

    def forward(self, words):
        emb = self.embedding(words)
        emb = emb.permute(1,0)
        conv_output = self.conv_layer(emb)
        relu = F.relu(conv_output)
        kmax = self.kmax_pooling(relu, 1, 1).squeeze()
        out = self.linear(kmax)
        return out
        
model = CNN(n_words=n_words, emb_size=EMBEDDING_SIZE, num_filters=64, window_size=2, ntags=n_class)

epochs = 20
eta = 0.05 
optimizer = torch.optim.SGD(model.parameters(), lr=eta)

for i in range(epochs):
    random.shuffle(train)
    train_loss = 0.0
    for words, tag in train:
        pred = model(torch.tensor(words))
        loss = F.cross_entropy(pred.reshape(1,-1), torch.tensor(tag).reshape(1))
        loss.backward()
        train_loss += loss
        optimizer.step()
        optimizer.zero_grad()

    print("iter %r: avg. train loss=%.4f" % (i, train_loss / len(train)))

iter 0: avg. train loss=1.1457
iter 1: avg. train loss=0.9828
iter 2: avg. train loss=0.9686
iter 3: avg. train loss=0.9581
iter 4: avg. train loss=0.9422
iter 5: avg. train loss=0.9588
iter 6: avg. train loss=0.9332
iter 7: avg. train loss=0.9199
iter 8: avg. train loss=0.9176
iter 9: avg. train loss=0.9300
iter 10: avg. train loss=0.8946
iter 11: avg. train loss=0.9203
iter 12: avg. train loss=0.9072
iter 13: avg. train loss=0.8523
iter 14: avg. train loss=0.8681
iter 15: avg. train loss=0.8774
iter 16: avg. train loss=0.8103
iter 17: avg. train loss=0.7365
iter 18: avg. train loss=0.8391
iter 19: avg. train loss=0.7542
